In [1]:
import jax
import jax.numpy as jnp
import flax
from flax import nnx

In [2]:
class MyModel(nnx.Module):
  def __init__(self, inSize: int, outSize: int, *, rngs: nnx.Rngs):
    intermediateSize = 64
    key = rngs.params()
    self.linear1 = nnx.Linear(inSize, intermediateSize, rngs=rngs)
    self.linear2 = nnx.Linear(intermediateSize, outSize, rngs=rngs)

  def __call__(self, x):
    x = self.linear1(x)
    x = jax.nn.relu(x)
    x = self.linear2(x)
    return x

In [3]:
rngs = nnx.Rngs(jax.random.key(0))
print(rngs)

myModel = MyModel(2, 2, rngs=rngs)
nnx.display(myModel)

Rngs( # RngState: 2 (12 B)
  default=RngStream( # RngState: 2 (12 B)
    key=RngKey( # 1 (8 B)
      value=Array((), dtype=key<fry>) overlaying:
      [0 0],
      tag="'default'"
    ),
    count=RngCount( # 1 (4 B)
      value=Array(0, dtype=uint32),
      tag="'default'"
    )
  )
)


In [4]:
graphDef, originalParams = nnx.split(myModel)
nnx.display(graphDef)
nnx.display(originalParams)

In [5]:
# Create new, different weights
newParams = jax.tree.map(lambda x: x*0, originalParams)

In [7]:
# Now, call the models with the different weights
originalModel = nnx.merge(graphDef, originalParams)
newModel = nnx.merge(graphDef, newParams)

nnx.display(originalModel)
nnx.display(newModel)

fakeData = jnp.array([0.1, -0.1])
print(originalModel(fakeData))
print(newModel(fakeData))

[0.09961922 0.03718882]
[0. 0.]
